In [1]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from models import model_dict
from dataset.cifar100 import *
from torchsummary import summary
from models.resnet import ResNet
import torch.optim as optim
import time
import argparse
import matplotlib.pyplot as plt
from munch import Munch

In [2]:
train_batch_size = 128
test_batch_size = 100
n_cls = 100
num_workers = 1
total_epoch = 240
learning_rate = 0.05
lr_decay_epoch = [150, 180, 210]
lr_decay_rate = 0.1
weight_decay = 5e-4
momentum = 0.9
print_freq = 100

In [3]:
# parser = argparse.ArgumentParser('argument for training')
# parser.add_argument('--model_t', type=str, default='resnet8', choices=['resnet8', 'resnet14', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110',
#                                  'resnet8x4', 'resnet32x4', 'wrn_16_1', 'wrn_16_2', 'wrn_40_1', 'wrn_40_2',
#                                  'vgg8', 'vgg11', 'vgg13', 'vgg16', 'vgg19', 'ResNet50',
#                                  'MobileNetV2', 'ShuffleV1', 'ShuffleV2'])
# parser.add_argument('--path_t', type=str, default=None)
# parser.add_argument('--model_s', type=str, default='resnet8', choices=['resnet8', 'resnet14', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110',
#                                  'resnet8x4', 'resnet32x4', 'wrn_16_1', 'wrn_16_2', 'wrn_40_1', 'wrn_40_2',
#                                  'vgg8', 'vgg11', 'vgg13', 'vgg16', 'vgg19', 'ResNet50',
#                                  'MobileNetV2', 'ShuffleV1', 'ShuffleV2'])
# parser.add_argument('--cuda_visible_devices', type=int, default=0)
# parser.add_argument('--trial', type=int, default=0)
# parser.add_argument('--balance', type=float, default=.2)
# opt = parser.parse_args()


In [4]:
opt = {'model_t': 'resnet32x4',
       'model_s': 'resnet8x4',
       'path_t': './save/models/resnet32x4_vanilla/ckpt_epoch_240.pth',
       'cuda_visible_devices': 0,
       'trial': 0,
       'balance': .2,
       'temp': 3.0}

opt = Munch(opt)

In [5]:
selection_dict = {
    'wrn401': ['wrn_40_1', './save/models/wrn_40_1_cifar100_lr_0.05_decay_0.0005_trial_0/wrn_40_1_best.pth',
            'wrn_16_1', 1, 1],
    'resnet324': ['resnet32x4', './save/models/resnet32x4_cifar100_lr_0.05_decay_0.0005_trial_0/resnet32x4_best.pth',
            'resnet8x4', 2, 1],
    'vgg19': ['vgg19', ' ./save/models/vgg19_cifar100_lr_0.05_decay_0.0005_trial_0/vgg19_best.pth',
            'vgg11', 2, 1],
    'resnet110_20': ['resnet110', ' ./save/models/resnet110_cifar100_lr_0.05_decay_0.0005_trial_0/resnet110_best.pth',
            'resnet20', 2, 1],
    'resnet110_32:':['resnet110', ' ./save/models/resnet110_cifar100_lr_0.05_decay_0.0005_trial_0/resnet110_best.pth',
            'resnet32', 3, 1]
}

is_pycharm = False
if is_pycharm:
    options = list(selection_dict.keys())
    arguments = selection_dict[options[1]]
    opt.model_t = arguments[0]
    opt.path_t = arguments[1]
    opt.model_s = arguments[2]
    opt.cuda_visible_devices = arguments[3]
    opt.trial = arguments[4]


In [6]:
print(opt)
os.environ['CUDA_VISIBLE_DEVICES'] = str(opt.cuda_visible_devices)

Munch({'model_t': 'resnet32x4', 'model_s': 'resnet8x4', 'path_t': './save/models/resnet32x4_vanilla/ckpt_epoch_240.pth', 'cuda_visible_devices': 0, 'trial': 0, 'balance': 0.2, 'temp': 3.0})


In [7]:
balance = opt.balance
model_t = model_dict[opt.model_t](num_classes=100)
model_s = model_dict[opt.model_s](num_classes=100)

In [8]:
path_t = opt.path_t
trial = 0
r = 1
a = 0
b = 1
p = 1
d = 1
kd_T = 4

In [9]:
model_t.load_state_dict(torch.load(path_t)['model'])
model_t.eval()

ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (b

In [10]:
model_t.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[ 6.7851e-02,  3.8761e-02,  6.4781e-02],
                        [ 3.1886e-02, -5.8565e-02,  2.9963e-02],
                        [ 6.5455e-02,  3.0583e-02,  6.5387e-02]],
              
                       [[ 2.0844e-02, -4.1083e-02,  3.6688e-02],
                        [-3.3501e-02, -1.7268e-01, -3.5568e-02],
                        [ 3.4867e-02, -3.2272e-02,  2.5880e-02]],
              
                       [[-1.9611e-02, -5.3734e-02, -2.6783e-02],
                        [-6.5794e-02, -1.6133e-01, -9.5393e-02],
                        [-3.9855e-02, -9.1401e-02, -5.2404e-02]]],
              
              
                      [[[ 3.2476e-01,  5.1935e-02, -2.6330e-01],
                        [ 5.7078e-01, -1.1742e-02, -6.2792e-01],
                        [ 4.0617e-01,  4.4851e-02, -3.7814e-01]],
              
                       [[-1.7055e-01, -6.8514e-03,  1.3657e-01],
                        [-1.5764e-01,  8.0631

In [11]:
# rand_value = torch.rand((2, 3, 32, 32))
# feature, logit = model_s(rand_value, is_feat=True, preact=True)
train_loader, val_loader, n_data = get_cifar100_dataloaders(batch_size=train_batch_size,
                                                                        num_workers=num_workers,
                                                                        is_instance=True)
if torch.cuda.is_available():
    model_s.cuda()
    model_t.cuda()

Files already downloaded and verified
Files already downloaded and verified


In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(epoch, learning_rate, lr_decay_epochs,  optimizer):
    """Sets the learning rate to the initial LR decayed by decay rate every steep step"""
    steps = np.sum(epoch > np.asarray(lr_decay_epochs))
    if steps > 0:
        new_lr = learning_rate * (lr_decay_rate ** steps)
        for param_group in optimizer.param_groups:
            param_group['lr'] = new_lr

In [13]:
class SepConv(nn.Module):
    def __init__(self, channel_in, channel_out, kernel_size=3, stride=2, padding=1, affine=True):
        super(SepConv, self).__init__()
        self.op = nn.Sequential(
            nn.Conv2d(channel_in, channel_in, kernel_size=kernel_size, stride=stride, padding=padding, groups=channel_in, bias=False),
            nn.Conv2d(channel_in, channel_in, kernel_size=1, padding=0, bias=False),
            nn.BatchNorm2d(channel_in, affine=affine),
            nn.ReLU(inplace=False),
            nn.Conv2d(channel_in, channel_in, kernel_size=kernel_size, stride=1, padding=padding, groups=channel_in, bias=False),
            nn.Conv2d(channel_in, channel_out, kernel_size=1, padding=0, bias=False),
            nn.BatchNorm2d(channel_out, affine=affine),
            nn.ReLU(inplace=False),
        )

    def forward(self, x):
        return self.op(x)


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [14]:
class TempDistiller(nn.Module):
    def __init__(self, in_channels, in_resolution):
        super(TempDistiller, self).__init__()
        self.module_list = []
        temp = in_resolution
        while temp > 4:
            self.module_list.append(SepConv(
                channel_in= in_channels,
                channel_out= in_channels * 2
            ))
            print(temp)
            in_channels *= 2
            temp /= 2
        self.avg_pool = nn.AvgPool2d(4, 4)
        self.module_list = nn.Sequential(*self.module_list)
        self.feat_num = 100
        self.fc = nn.Linear(in_channels, 100)


    def forward(self, in_feat):
        out = self.module_list(in_feat)
        out = self.avg_pool(out)
        feature = out.view(out.size(0), -1)
        out_fc = self.fc(feature)
        return out_fc, feature

class SimpleMLP(nn.Module):
    def __init__(self, input_size, num_classes=100):
        super(SimpleMLP, self).__init__()

        self.simple = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.BatchNorm1d(128),
            nn.ReLU6(),
            nn.Linear(128, num_classes)
        )

    def forward(self, input):
        return self.simple(input)

# Knowledge distillation Idea
class AlternativeL2Loss(nn.Module):
    def __init__(self):
        super(AlternativeL2Loss, self).__init__()
        pass

    def forward(self, source, target, margin=2):
        loss = ((source + margin) ** 2 * ((source > -margin) & (target <= 0)).float() +
                (source - margin) ** 2 * ((source <= margin) & (target > 0)).float())
        return torch.abs(loss).mean()
    
def KDLoss(outputs, targets):
    log_softmax_outputs = F.log_softmax(outputs/opt.temp, dim=1)
    softmax_targets = F.softmax(targets/opt.temp, dim=1)
    return -(log_softmax_outputs * softmax_targets).sum(dim=1).mean()

In [15]:
rand_ = torch.rand((1, 3, 32, 32), dtype=torch.float32).cuda()
out_feat, out_x = model_t(rand_, is_feat=True, preact=True)

conv_distiller = nn.ModuleList([TempDistiller(f.size(1), f.size(2)).cuda() for f in out_feat[:-1]])
MLP = SimpleMLP(out_feat[-1].size(1), 100).cuda()
# MLP2 = SimpleMLP(out_feat[-1].size(1), 100).cuda()
trainable_models = nn.ModuleList([])
trainable_models.MLP = MLP
trainable_models.model_s = model_s
trainable_models.conv_distiller = conv_distiller
optimizer = optim.SGD(trainable_models.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
criterion_CE = nn.CrossEntropyLoss()
criterion_CE_1 = nn.CrossEntropyLoss()
criterion_FM = AlternativeL2Loss()
criterion_AB = AlternativeL2Loss()
criterion_LS = nn.BCEWithLogitsLoss()
criterion_MSE = nn.MSELoss()

best_accuracy = -1
best_mlp_accuracy = -1
best_ensemble_acc = -1
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
if is_pycharm:
    loss_ce_list = []
    loss_kd_list = []
    loss_simple_list = []

32
16.0
8.0
32
16.0
8.0
16
8.0
8


In [16]:
print("Batch size = {}".format(2))
for i, o in enumerate(out_feat[:-1]):
    print(o.shape)
    print(conv_distiller[i](o)[0].shape)
print(out_x.shape)

Batch size = 2
torch.Size([1, 32, 32, 32])
torch.Size([1, 100])
torch.Size([1, 64, 32, 32])
torch.Size([1, 100])
torch.Size([1, 128, 16, 16])
torch.Size([1, 100])
torch.Size([1, 256, 8, 8])
torch.Size([1, 100])
torch.Size([1, 100])


In [17]:
# merge all trainable models into trainable_models
trainable_models.cuda()

ModuleList(
  (MLP): SimpleMLP(
    (simple): Sequential(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6()
      (3): Linear(in_features=128, out_features=100, bias=True)
    )
  )
  (model_s): ResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [ ]:
for epoch in range(1, total_epoch+1):
    adjust_learning_rate(epoch, learning_rate, lr_decay_epoch, optimizer)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    top1_ensemble, top5_ensemble = AverageMeter(), AverageMeter()
    loss_ce_ = AverageMeter()
    loss_kd_ = AverageMeter()
    loss_simple_ = AverageMeter()


#     MLP.train() # for final_feature
#   #  MLP2.train()
#     model_s.train()
#     conv_distiller.train()
    trainable_models.train()
    end = time.time()

    avg_loss_ce = 0
    avg_loss_simple = 0
    count = 0

    similarity_list = []
    loss_list = []

    for idx, data in enumerate(train_loader):
        loss_batch_list = []
        input, target, index = data
        input = input.float()
        # target = target.float()
        optimizer.zero_grad()

        data_time.update(time.time() - end)
        if torch.cuda.is_available():
            input = input.cuda()
            target = target.cuda()

        with torch.no_grad():
            feat_t, logit_t = model_t(input, is_feat=True, preact=True)
        feat_s, logit_s = trainable_models.model_s(input, is_feat=True, preact=True)

        cc = 0
        sub_loss = 0
        sub_loss_2 = 0
        sub_loss_3 = 0

        loss_1 = torch.FloatTensor([0.]).cuda()
        loss_2 = torch.FloatTensor([0.]).cuda()
        loss_3 = torch.FloatTensor([0.]).cuda()
        loss_4 = torch.FloatTensor([0.]).cuda()
        ensemble = torch.zeros(size=(input.size(0), n_cls)).cuda() # ensemble init (avg)
        
        # FitNet + SD part
        for conv, fs, ft in zip(trainable_models.conv_distiller, feat_s, feat_t):
            conv_logit_t, conv_feature_t = conv(ft.detach())
            conv_logit_s, conv_feature_s = conv(fs)
            ensemble += conv_logit_s # ensemble        
#             loss_1 += 0.5 * F.kl_div(F.log_softmax(conv_logit_t / kd_T, dim=1), F.softmax(logit_t.detach() / kd_T, dim=1), size_average=False) * (kd_T ** 2) / conv_logit_t.size(0)
#             loss_2 += 2 * F.kl_div(F.log_softmax(conv_logit_s / kd_T, dim=1), F.softmax(conv_logit_t.detach() / kd_T, dim=1), size_average=False) * (kd_T ** 2) / conv_logit_s.size(0)
            
            # final feature map KD (final teacher - student_feature_logit_i)
            loss_1 += KDLoss(conv_logit_s, logit_t.detach()) 
    
            # FitNet MSE (feature teacher - feature student)
            loss_3 += criterion_MSE(fs, ft.detach())
            
            # CE by label (target - student_feature_logit_i)
            loss_4 += criterion_CE(conv_logit_s, target)
            cc += 1

        ensemble /= cc # ensemble avg
        loss_batch_list.append(loss_1.item() / cc)
#         loss_batch_list.append(loss_2.item() / cc)
        loss_batch_list.append(loss_3.item() / cc)

        simple_out_t = trainable_models.MLP(feat_t[-1].detach())
        simple_out_s = trainable_models.MLP(feat_s[-1])

        similarity_list.append \
            ([cos(simple_out_t, logit_t).mean().item(),
              cos(simple_out_s, simple_out_t).mean().item(),
              cos(simple_out_s, logit_s).mean().item(),
              cos(logit_s, logit_t).mean().item()])
        
         #
        # if epoch == 10:
        #     print(simple_out_t)
        #     print(simple_out_s)
        #     exit()

        loss_simple = ((balance * F.kl_div(F.log_softmax(simple_out_t / kd_T,dim=1), F.softmax(logit_t.detach() / kd_T, dim=1),size_average=False) +
                       (1 - balance) * F.kl_div(F.log_softmax(simple_out_s / kd_T, dim=1),
                                                F.softmax(simple_out_t.detach() / kd_T, dim=1), size_average=False))) * (
                                  kd_T ** 2) / logit_s.size(0)

       # loss_simple = torch.FloatTensor([0.]).cuda()

        loss_batch_list.append(loss_simple.item())
        # loss_simple = (balance * criterion_AB(simple_out_t, logit_t.detach()) +
        #
        #              (1 - balance) * F.kl_div(F.log_softmax(simple_out_s / kd_T, dim=1), F.softmax(simple_out_t.detach() / kd_T, dim=1), size_average=False)) * (kd_T ** 2) / logit_s.size(0) \
        #             + (1 - balance) * F.kl_div(F.log_softmax(logit_s / kd_T, dim=1), F.softmax(simple_out_s / kd_T, dim=1), size_average=False) * (kd_T ** 2) / logit_s.size(0)
        # loss_simple = balance * criterion_CE2(simple_out_t / kd_T, logit_t.detach() / kd_T) + (1 - balance) * criterion_CE2(simple_out_s / kd_T, simple_out_t.detach())
        # loss_simple_ce = criterion_CE_1(simple_out_t, target) + criterion_CE_1(simple_out_s, target) # idea 5
       # loss_simple_ce = criterion_CE_1(simple_out_s, target) # idea 6
        # loss_simple.backward(retain_graph=True)
        loss_kd = (kd_T ** 2) * F.kl_div(F.log_softmax(logit_s / kd_T, dim=1), F.softmax(logit_t.detach() / kd_T, dim=1), size_average=False) / logit_s.size(0)

        loss_ce = criterion_CE(logit_s, target)
        loss_batch_list.append(loss_ce.item())
        loss_fm = criterion_AB(logit_s, logit_t.detach())
        loss_batch_list.append(loss_fm.item())

        acc_1, acc_5 = accuracy(logit_s, target, topk=(1, 5))
        acc_1_ensemble, acc_5_ensemble = accuracy(ensemble, target, topk=(1, 5))
        top1.update(acc_1[0], input.size(0))
        top5.update(acc_5[0], input.size(0))
        top1_ensemble.update(acc_1_ensemble[0], input.size(0))
        top5_ensemble.update(acc_5_ensemble[0], input.size(0))

        count += 1

        loss = loss_1 + loss_3 + loss_4 + loss_simple + loss_ce * 0.1 + loss_fm + loss_kd * 0.9
        losses.update(loss.item())
        loss.backward()
        optimizer.step()
        batch_time.update(time.time() - end)
        end = time.time()
        loss_list.append(loss_batch_list)

#         if idx % print_freq == 0:
#             print('Epoch: [{0}][{1}/{2}]\t'
#                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                   'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
#                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                   'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
#                   'Acc@5 {top5.val:.3f} ({top5.avg:.3f})\t'
#                   'Acc_Ensemble@1 {top1_ens.val:.3f} ({top1_ens.avg:.3f})\t'
#                   'Acc_Ensemble@5 {top5_ens.val:.3f} ({top5_ens.avg:.3f})'.format(
#                 epoch, idx, len(train_loader), batch_time=batch_time,
#                 data_time=data_time, loss=losses, top1=top1, top5=top5, top1_ens=top1_ensemble, top5_ens=top5_ensemble))
#             sys.stdout.flush()

    print('Epoch: {}, [Train]* Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f} Acc_Ensemble@1 {top1_ens.avg:.3f} Acc_Ensemble@5 {top5_ens.avg:.3f}'
          .format(epoch, top1=top1, top5=top5, top1_ens=top1_ensemble, top5_ens=top5_ensemble))
    print('cos(simple_out_t, logit_s) | cos(simple_out_t, simple_out_s) | cos(simple_out_s, logit_s) | cos(logit_s, logit_t)')
    print(np.array(similarity_list).mean(axis=0))
    loss_list = np.array(loss_list)
    avg_loss_list = np.sum(loss_list, axis=0) / count

    print("Conv(L_T, C_T): %.4f Conv_FM: %.4f,  Loss(simple): %.4f CE: %.4f FM: %.4f" %
          (avg_loss_list[0], avg_loss_list[1], avg_loss_list[2],avg_loss_list[3],avg_loss_list[4]))
    # if False:
    #
    # #if is_pycharm and (epoch % 5 == 0 and epoch > 30) or epoch == 3:
    #     print(loss_ce_list)
    #     plt.plot(list(range(len(loss_ce_list))), loss_ce_list, label='ce')
    #     plt.plot(list(range(len(loss_kd_list))), loss_kd_list, label='kd')
    #     plt.plot(list(range(len(loss_simple_list))), loss_simple_list, label='ours')
    #     plt.legend()
    #     plt.show()

    trainable_models.eval()
    similarity_t_list = []
    with torch.no_grad():
        val_batch_time = AverageMeter()
        val_top1 = AverageMeter()
        val_top5 = AverageMeter()
        val_losses = AverageMeter()
        val_top1_ensemble = AverageMeter()
        val_top5_ensemble = AverageMeter()
        mlp_top1 = AverageMeter()
        mlp_top5 = AverageMeter()
        for idx, (input, target) in enumerate(val_loader):
            input = input.float().cuda()
            target = target.cuda()

            val_feat_s, output = trainable_models.model_s(input, is_feat=True, preact=True)
            loss = criterion_CE(F.softmax(output), target)
            simple_val_out_s = trainable_models.MLP(val_feat_s[-1])
            ensemble = torch.zeros(size=[input.size(0), n_cls]).cuda()
            # ensemble
            cc = 0
            for conv, fs in zip(trainable_models.conv_distiller, val_feat_s):
                conv_logit_s, conv_feature_s = conv(fs)
                ensemble += conv_logit_s # ensemble        
                cc += 1
            ensemble /= cc # ensemble avg


          #  print(cos(simple_val_out_s, output))
            similarity_t_list.append(cos(simple_val_out_s, output).mean().item())
            val_acc_1, val_acc_5 = accuracy(output, target, topk=(1, 5))
            val_acc_1_ensemble, val_acc_5_ensemble = accuracy(ensemble, target, topk=(1, 5))
            mlp_val_acc_1, mlp_val_acc_5 = accuracy(simple_val_out_s, target, topk = (1, 5))
            val_top1.update(val_acc_1[0], input.size(0))
            val_top5.update(val_acc_5[0], input.size(0))
            val_top1_ensemble.update(val_acc_1_ensemble[0], input.size(0))
            val_top5_ensemble.update(val_acc_5_ensemble[0], input.size(0))
            mlp_top1.update(mlp_val_acc_1[0], input.size(0))
            mlp_top5.update(mlp_val_acc_5[0], input.size(0))

#             if idx % print_freq == 0:
#                 print('Test: [{0}/{1}]\t'
#                       'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                       'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                       'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
#                       'Acc@5 {top5.val:.3f} ({top5.avg:.3f})\t'
#                       'Acc_Ensemble@1 {top1_ens.val:.3f} ({top1_ens.avg:.3f})\t'
#                       'Acc_Ensemble@5 {top5_ens.val:.3f} ({top5_ens.avg:.3f})\t'
#                       'AccMLP@1 {mlp_top1.val:.3f} ({mlp_top1.avg:.3f})\t'
#                       'AccMLP@5 {mlp_top5.val:.3f} ({mlp_top5.avg:.3f})'.format(
#                        idx, len(val_loader), batch_time=batch_time, loss=losses,
#                        top1=val_top1, top5=val_top5, top1_ens=val_top1_ensemble, top5_ens=val_top5_ensemble, mlp_top1=mlp_top1, mlp_top5=mlp_top5))
    if best_accuracy < val_top1.avg:
        best_accuracy = val_top1.avg
    if best_mlp_accuracy < mlp_top1.avg:
        best_mlp_accuracy = mlp_top1.avg
    best_ensemble_acc = max(best_ensemble_acc, val_top1_ensemble.avg)
    print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f} Acc_Ensemble@1 {top1_ens.avg:.3f} Acc_Ensemble@5 {top5_ens.avg:.3f} mlpAcc@1 {mlp_top1.avg:.3f} mlpAcc@5 {mlp_top5.avg:.3f} Best_accuracy %.4f Best_mlp_accuracy %.4f'
          .format(top1=val_top1, top5=val_top5, top1_ens=val_top1_ensemble, top5_ens=val_top5_ensemble, mlp_top1=mlp_top1, mlp_top5=mlp_top5) % (best_accuracy, best_mlp_accuracy))
    print("cos(simple_out_s, logit_s)", sum(similarity_t_list) / len(similarity_t_list))



/home/anaconda/lib/python3.7/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch: 1, [Train]* Acc@1 7.496 Acc@5 25.812 Acc_Ensemble@1 10.792 Acc_Ensemble@5 32.218
cos(simple_out_t, logit_s) | cos(simple_out_t, simple_out_s) | cos(simple_out_s, logit_s) | cos(logit_s, logit_t)
[0.79344846 0.40682369 0.68461851 0.30637269]
Conv(L_T, C_T): 4.3994 Conv_FM: 0.4470,  Loss(simple): 4.4105 CE: 4.0391 FM: 3.6029


/home/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:181: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


 * Acc@1 8.920 Acc@5 30.370 Acc_Ensemble@1 14.650 Acc_Ensemble@5 38.960 mlpAcc@1 10.730 mlpAcc@5 33.460 Best_accuracy 8.9200 Best_mlp_accuracy 10.7300
cos(simple_out_s, logit_s) 0.81820366071288
Epoch: 2, [Train]* Acc@1 15.424 Acc@5 42.494 Acc_Ensemble@1 23.066 Acc_Ensemble@5 52.882
cos(simple_out_t, logit_s) | cos(simple_out_t, simple_out_s) | cos(simple_out_s, logit_s) | cos(logit_s, logit_t)
[0.90342333 0.49981905 0.83583524 0.41672886]
Conv(L_T, C_T): 4.2260 Conv_FM: 0.4022,  Loss(simple): 5.9184 CE: 3.5428 FM: 3.4730
 * Acc@1 16.770 Acc@5 45.520 Acc_Ensemble@1 24.620 Acc_Ensemble@5 55.860 mlpAcc@1 19.590 mlpAcc@5 49.610 Best_accuracy 16.7700 Best_mlp_accuracy 19.5900
cos(simple_out_s, logit_s) 0.832619306767822
Epoch: 3, [Train]* Acc@1 23.482 Acc@5 55.420 Acc_Ensemble@1 33.216 Acc_Ensemble@5 66.320
cos(simple_out_t, logit_s) | cos(simple_out_t, simple_out_s) | cos(simple_out_s, logit_s) | cos(logit_s, logit_t)
[0.90801798 0.5599986  0.85991647 0.49696405]
Conv(L_T, C_T): 4.0959 Co

#### Teacher performance check: 75.4

In [ ]:
with torch.no_grad():
    val_batch_time = AverageMeter()
    val_top1 = AverageMeter()
    val_top5 = AverageMeter()
    val_losses = AverageMeter()
    mlp_top1 = AverageMeter()
    mlp_top5 = AverageMeter()
    for idx, (input, target) in enumerate(val_loader):
        input = input.float().cuda()
        target = target.cuda()

        val_feat_t, output = model_t(input, is_feat=True, preact=True)
        loss = criterion_CE(F.softmax(output), target)
        simple_val_out_t = MLP(val_feat_t[-1])

      #  print(cos(simple_val_out_s, output))
        similarity_t_list.append(cos(simple_val_out_t, output).mean().item())
        val_acc_1, val_acc_5 = accuracy(output, target, topk=(1, 5))
        mlp_val_acc_1, mlp_val_acc_5 = accuracy(simple_val_out_t, target, topk = (1, 5))
        val_top1.update(val_acc_1[0], input.size(0))
        val_top5.update(val_acc_5[0], input.size(0))
        mlp_top1.update(mlp_val_acc_1[0], input.size(0))
        mlp_top5.update(mlp_val_acc_5[0], input.size(0))

        if idx % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Acc@5 {top5.val:.3f} ({top5.avg:.3f})\t'
                  'AccMLP@1 {mlp_top1.val:.3f} ({mlp_top1.avg:.3f})\t'
                  'AccMLP@5 {mlp_top5.val:.3f} ({mlp_top5.avg:.3f})'.format(
                   idx, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=val_top1, top5=val_top5, mlp_top1=mlp_top1, mlp_top5=mlp_top5))
if best_accuracy < val_top1.avg:
    best_accuracy = val_top1.avg
if best_mlp_accuracy < mlp_top1.avg:
    best_mlp_accuracy = mlp_top1.avg
print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f} mlpAcc@1 {mlp_top1.avg:.3f} mlpAcc@5 {mlp_top5.avg:.3f} Best_accuracy %.4f Best_mlp_accuracy %.4f'
      .format(top1=val_top1, top5=val_top5, mlp_top1=mlp_top1, mlp_top5=mlp_top5) % (best_accuracy, best_mlp_accuracy))
print("cos(simple_out_s, logit_s)", sum(similarity_t_list) / len(similarity_t_list))


In [ ]:
class RKDLoss(nn.Module):
    """Relational Knowledge Disitllation, CVPR2019"""
    def __init__(self, w_d=25, w_a=50):
        super(RKDLoss, self).__init__()
        self.w_d = w_d
        self.w_a = w_a

    def forward(self, f_s, f_t):
        student = f_s.view(f_s.shape[0], -1)
        teacher = f_t.view(f_t.shape[0], -1)

        # RKD distance loss
        with torch.no_grad():
            t_d = self.pdist(teacher, squared=False)
            mean_td = t_d[t_d > 0].mean()
            t_d = t_d / mean_td

        d = self.pdist(student, squared=False)
        mean_d = d[d > 0].mean()
        d = d / mean_d

        loss_d = F.smooth_l1_loss(d, t_d)

        # RKD Angle loss
        with torch.no_grad():
            td = (teacher.unsqueeze(0) - teacher.unsqueeze(1))
            norm_td = F.normalize(td, p=2, dim=2)
            t_angle = torch.bmm(norm_td, norm_td.transpose(1, 2)).view(-1)

        sd = (student.unsqueeze(0) - student.unsqueeze(1))
        norm_sd = F.normalize(sd, p=2, dim=2)
        s_angle = torch.bmm(norm_sd, norm_sd.transpose(1, 2)).view(-1)

        loss_a = F.smooth_l1_loss(s_angle, t_angle)

        loss = self.w_d * loss_d + self.w_a * loss_a

        return loss

    @staticmethod
    def pdist(e, squared=False, eps=1e-12):
        e_square = e.pow(2).sum(dim=1)
        prod = e @ e.t()
        res = (e_square.unsqueeze(1) + e_square.unsqueeze(0) - 2 * prod).clamp(min=eps)

        if not squared:
            res = res.sqrt()

        res = res.clone()
        res[range(len(e)), range(len(e))] = 0
        return res

